# Take a picture and run the cloud model on it

## Prerequisites and cloud credential setup

In [ ]:
import os
import glob

from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageUrlCreateEntry

from picamera import PiCamera
from time import sleep
ENDPOINT = "https://westus2.api.cognitive.microsoft.com"

# Replace with a valid key
training_key = "YOURTRAININGKEY"
prediction_key = "YOURPREDICTIONKEY"
prediction_resource_id = "YOURRESOURCEID"

from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient

projectid="YOURPROJECTID"
# Now there is a trained endpoint that can be used to make a prediction
predictor = CustomVisionPredictionClient(prediction_key, endpoint=ENDPOINT)
trainer = CustomVisionTrainingClient(training_key,endpoint=ENDPOINT)


## Find a published iteration

In [ ]:
iterations = trainer.get_iterations(projectid);
for iteration in iterations:
    print(iteration.name + "\t" + iteration.id + "\t" + iteration.status, end=' ' )
    if (iteration.publish_name):
        print("published under "+ iteration.publish_name, end=' ')
        publish_iteration_name = iteration.publish_name
    print(" ")
    
print("using "+ publish_iteration_name)


## Init camera

In [ ]:

camera = PiCamera()

camera.resolution =(640,480)


print ("starting camera")
camera.start_preview()
sleep(5)


## send the picture to the cloud for analysis and print the result

In [ ]:
print ("capture")
camera.capture('/home/pi/camera/image.jpg')


print ("analyzing")
with open('/home/pi/camera/image.jpg',mode="rb") as test_data:
    results = predictor.classify_image(projectid, publish_iteration_name, test_data.read())
    for prediction in results.predictions:
        print ("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100))
        



## Exit

In [ ]:
camera.stop_preview()